In [1]:
import os

In [2]:
%pwd

'c:\\Users\\steva\\Documents\\Projects\\data-science\\deep-learning-pipeline\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\steva\\Documents\\Projects\\data-science\\deep-learning-pipeline'

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    kaggle_dataset:  str
    local_data_file: Path
    unzip_dir: Path





In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories



In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath =  PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            kaggle_dataset= config.kaggle_dataset,
            local_data_file=config.local_data_file,
            unzip_dir =  config.unzip_dir,

        )

        return data_ingestion_config
 



In [8]:
import os
import urllib.request as request
import zipfile
import kaggle
import json
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
from pathlib import Path

In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def setup_kaggle_auth(self):
        kaggle_json_path = os.path.join(os.path.expanduser('~'), '.kaggle', 'kaggle.json')
        if not os.path.exists(kaggle_json_path):
            raise FileNotFoundError(f"Kaggle API credentials not found at {kaggle_json_path}. Please set up your Kaggle API credentials.")
        
        # Memeriksa izin file
        if os.name != 'nt':  # Untuk sistem non-Windows
            if os.stat(kaggle_json_path).st_mode & 0o777 != 0o600:
                os.chmod(kaggle_json_path, 0o600)
                logger.warning(f"Changed permissions of {kaggle_json_path} to 0600 for security.")
        
        # Menyiapkan environment variables untuk Kaggle API
        with open(kaggle_json_path, 'r') as f:
            kaggle_cred = json.load(f)
        os.environ['KAGGLE_USERNAME'] = kaggle_cred['username']
        os.environ['KAGGLE_KEY'] = kaggle_cred['key']
        
        logger.info("Kaggle authentication setup completed.")

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            if self.config.kaggle_dataset:
                self.setup_kaggle_auth()  # Menyiapkan autentikasi sebelum mengunduh
                kaggle.api.dataset_download_files(
                    self.config.kaggle_dataset,
                    path=os.path.dirname(self.config.local_data_file),
                    unzip=False
                )
                logger.info(f"Dataset downloaded from Kaggle: {self.config.kaggle_dataset}")
            elif self.config.source_URL:
                filename, headers = request.urlretrieve(
                    url = self.config.source_URL,
                    filename = self.config.local_data_file
                )
                logger.info(f"{filename} download! with following info: \n{headers}")
            else:
                logger.info("No Kaggle dataset or source URL specified. Using local file.")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-05 20:47:57,106] - INFO - yaml file: config\config.yaml loaded successfully
[2024-10-05 20:47:57,107] - INFO - yaml file: params.yaml loaded successfully
[2024-10-05 20:47:57,109] - INFO - created directory at: artifacts
[2024-10-05 20:47:57,109] - INFO - created directory at: artifacts/data_ingestion
[2024-10-05 20:47:57,109] - INFO - File already exists of size: ~ 5440432 KB
